# Solve a Basic Model

In this example, we explain the basic functions of the linopy `Model` class. First, we are setting up a very simple linear optimization model, given by 

Minimize:
$$ x + 2y $$
      
subject to:

$$ x \ge 0 $$
$$y \ge 0 $$
$$3x + 7y \ge 10 $$
$$5x + 2y \ge 3 $$

### Initializing a `Model`

The Model class in Linopy is a fundamental part of the library. It serves as a container for all the relevant data associated with a linear optimization problem. This includes variables, constraints, and the objective function.

In [ ]:
from linopy import Model

m = Model()

This creates a new Model object, which you can then use to define your optimization problem.



### Adding variables

Variables in a linear optimization problem represent the decision variables. A variable can always be assigned with a lower and an upper bound. In our case, both `x` and `y` have a lower bound of zero (default is unbouded). In Linopy, you can add variables to a Model using the `add_variables` method:

In [ ]:
x = m.add_variables(lower=0, name="x")
y = m.add_variables(lower=0, name="y");

`x` and `y` are linopy variables of the class `linopy.Variable`. Each of them contain all relevant information that define it. The `name` parameter is optional but can be useful for referencing the variables later.

In [ ]:
x

Since both `x` and `y` are scalar variables (meaning they don't have any dimensions), their underlying data contain only one optimization variable each.  

### Adding Constraints

Constraints define the feasible region of the optimization problem. They consist of the left hand side (lhs) and the right hand side (rhs). The first constraint that we want to write down is 
$3x + 7y >= 10$, which we write out exactly in the mathematical way

In [ ]:
3 * x + 7 * y >= 10

Note, we can also mix the constant and the variable expression, like this

In [ ]:
3 * x + 7 * y - 10 >= 0

... and linopy will automatically take over the separation of variables expression on the lhs, and constant values on the rhs.

The constraint is currently not assigned to the model. We assign it by calling the function `m.add_constraints`.

In [ ]:
m.add_constraints(3 * x + 7 * y >= 10)
m.add_constraints(5 * x + 2 * y >= 3);

## Adding the Objective 

The objective function defines what you want to optimize. You can set the objective function of a Model in Linopy using the add_objective method. For our example that would be

In [ ]:
m.add_objective(x + 2 * y)

## Solving the Model

Once you've defined your Model with variables, constraints, and an objective function, you can solve it using the `solve` method:

In [ ]:
m.solve(solver_name="highs")

The solution of the linear problem assigned to the variables under `solution` in form of a `xarray.Dataset`. 

In [ ]:
x.solution

In [ ]:
y.solution

### Solver Metrics

After solving, you can inspect performance metrics reported by the solver via `solver_metrics`. This includes solve time, objective value, and for MIP problems, the dual bound and MIP gap (available for most solvers.

In [ ]:
m.solver_metrics

SolverMetrics(solver_name='highs', solve_time=0.0019101661164313555, objective_value=2.862068965517241, dual_bound=0.0, mip_gap=inf)

Well done! You solved your first linopy model!